In [1]:
import json
from web3 import Web3
import os
import platform
from getpass import getpass
import random

In [255]:
operating_system = platform.system()
d = '/'
if operating_system == 'Windows':
    d = '\\'

ganache_url = "HTTP://127.0.0.1:7545"
web3 = Web3(Web3.HTTPProvider(ganache_url))

# This is our main account that controls everything
web3.eth.defaultAccount = web3.eth.accounts[0]

In [256]:
def getContract(filename):
    with open('..'+ d +'build'+ d +'contracts'+ d +filename) as contractFile:
        contractJson=json.load(contractFile)
        contractABI =contractJson['abi']
        contractAdd=web3.toChecksumAddress(contractJson['networks']['5777']['address']) 
        return contractABI, contractAdd



# Get the organization contract address
organizationContractABI, organizationContractAdd = getContract('Organization.json')	
organizationContract = web3.eth.contract(address=organizationContractAdd, abi=organizationContractABI)

#get the user contract address
userContractABI,userContractAdd = getContract('User.json')
userContract = web3.eth.contract(address=userContractAdd, abi=userContractABI)

# Get the accounts contract address
accountsContractABI,accountsContractAdd = getContract('Accounts.json')
accountsConract = web3.eth.contract(address=accountsContractAdd, abi=accountsContractABI)

loansContractABI, loansContractAddress= getContract('Loans.json')


In [257]:
# This should be changed to be a more robus way. We can give the loansContractAddress using the constructor
# but this has problems when doing this in the 2_deploy_contracts.js
userContract.functions.setLoansContractAddress(loansContractAddress).transact()
organizationContract.functions.setLoansContractAddress(loansContractAddress).transact()

HexBytes('0xdec3a0b0417a8edfbe597ccc1eb49bdd4c46c3a09d0c6b45ce5b499cd647c771')

HexBytes('0x52925bfd1a376ad5cb922111e0c7b77c67be7da7c94e5e894aac1a603cdc2899')

In [258]:
accountsConract.functions.add(web3.eth.accounts[1], False).transact()
accountsConract.functions.add(web3.eth.accounts[2], False).transact()
accountsConract.functions.add(web3.eth.accounts[5], True).transact()
accountsConract.functions.add(web3.eth.accounts[6], True).transact()

HexBytes('0xe8c28ab68968898f1bdb6723008d8c118d51b5a76d0412f3f7ba4fc97f830442')

HexBytes('0xf6284ceecf6bfee320cf074134a363ccf25b23c6e2efd142bdca72854b9a0b10')

HexBytes('0x6567e0033fbeaf589112a1dfb4e1fc2d588ef7ac951ccf86488df946c2d2b392')

HexBytes('0xcad30ff6948f4e12e940e8012b7ec4bd6c027489b6cc23fc373c9ec26f64b701')

In [259]:
def deleteUser():
    accountsConract.functions.deleteAccount(web3.eth.accounts[1]).transact()

def createLoan(_loanie, _loaner, _amount, _organizationContract):
    
    transaction = _organizationContract.functions.createLoan(_loanie, _amount 
    ).buildTransaction({
    'gas': 3000000,
    'gasPrice': web3.toWei('1', 'gwei'),
    'from': _loaner,
    'nonce': web3.eth.getTransactionCount(_loaner)
    }) 
    _privateKey = getpass("Enter the password of the organization: ")
    signed_txn = web3.eth.account.signTransaction(transaction, private_key=_privateKey)
    transaction_hash = web3.eth.sendRawTransaction(web3.toHex(signed_txn.rawTransaction))
    
    # _organizationContract.functions.createLoan(_loanie, _loaner, _amount).transact()
    return True

In [99]:
deleteUser()

In [260]:
def getPendingLoans(_userContract, _loanieAddress, _privateKey):
    transaction = _userContract.functions.getPendingLoans(
    ).buildTransaction({
    'gas': 70000,
    'gasPrice': web3.toWei('1', 'gwei'),
    'from': _loanieAddress,
    'nonce': web3.eth.getTransactionCount(_loanieAddress)
    }) 
    
    signed_txn = web3.eth.account.signTransaction(transaction, private_key=_privateKey)
    transaction_hash = web3.eth.sendRawTransaction(web3.toHex(signed_txn.rawTransaction))
    receipt = web3.eth.getTransactionReceipt(transaction_hash)
    rich_logs = _userContract.events.getAmounts().processReceipt(receipt)
    event_values = rich_logs[0]['args'] # Dictionary
    return event_values


In [265]:
# Suppose bank wants to create a loan
#loaner = input("Enter the loaner addrses: ")
lonaer = web3.eth.accounts[5]
loaner
### Check to see whether it is a user or organization address
loanerIndex = accountsConract.functions.getIndex(loaner).call()

'0x37594b8C8aC1D94457C86e30dE793099CE659817'

In [264]:
#loanie = input("Enter the loanie address: ")
loanie = web3.eth.accounts[2]
loanie
loanieIndex = accountsConract.functions.getIndex(loanie).call()

'0xEE4F5078d802207d68eE9668AE6b82FCFCF7e290'

## Get pending loans

In [269]:
# Get pending loans using 3 separate functions
def getPendingLoansList(_accountIndex, _userContract, _accountsContract, _loanieAddress):
    pendingLoans = [] # 'amount loanerAddress id'
    if _accountIndex != -1:
        loanieType = _accountsContract.functions.getType(_accountIndex).call()
        if not loanieType:
            privateKey = getpass("Enter your password: ")
            values = getPendingLoans(_userContract, _loanieAddress, privateKey)
            for i in range(len(values['_amounts'])):
                string = ''
                for key in values:
                    string += str(values[key][i]) + ' '
                pendingLoans.append(string)
    else:
        print("This account is not registered in our system.")
    return pendingLoans

In [270]:
getPendingLoansList(loanieIndex, userContract, accountsConract, loanie)

Enter your password: ········


['33351 0x37594b8C8aC1D94457C86e30dE793099CE659817 1589030343 ',
 '11029 0x37594b8C8aC1D94457C86e30dE793099CE659817 1589030344 ',
 '26884 0x37594b8C8aC1D94457C86e30dE793099CE659817 1589030345 ']

## Confirm or reject loans

In [271]:
# Confirm or reject loans
pendingLoans = getPendingLoansList(loanieIndex, userContract, accountsConract, loanie)
private_key = getpass('Enter your loanie password: ')
for pendingLoan in pendingLoans:
    print('for id: ' + pendingLoan.split(' ')[0])
    loanId = pendingLoan.split(' ')[2]
    choice = input('c/r?')
    if choice == 'c':
        transaction = userContract.functions.validateLoan(True,
        int(loanId)).buildTransaction({
        'gas': 70000,
        'gasPrice': web3.toWei('1', 'gwei'),
        'from': loanie,
        'nonce': web3.eth.getTransactionCount(loanie)
        }) 
        signed_txn = web3.eth.account.signTransaction(transaction, private_key=private_key)
        transaction_hash = web3.eth.sendRawTransaction(web3.toHex(signed_txn.rawTransaction))
    elif choice == 'r':
        transaction = userContract.functions.validateLoan(False,
        int(loanId)).buildTransaction({
        'gas': 70000,
        'gasPrice': web3.toWe8a64a997b793d9c84c1c34a8d9e6e607adffedb4193a823fcac9489e12f76c5di('1', 'gwei'),
        'from': loanie,
        'nonce': web3.eth.getTransactionCount(loanie)
        }) 
        signed_txn = web3.eth.account.signTransaction(transaction, private_key=private_key)
        transaction_hash = web3.eth.sendRawTransaction(web3.toHex(signed_txn.rawTransaction))
    else:
        print("Not recognized operation.")

Enter your password: ········
Enter your loanie password: ········
for id: 33351
c/r?c


ValueError: {'message': 'VM Exception while processing transaction: revert', 'code': -32000, 'data': {'0xeb8572e641f9e534252eab3893db4578655c6cbefc5a6d398991ba75182b3f04': {'error': 'revert', 'program_counter': 331, 'return': '0x'}, 'stack': 'RuntimeError: VM Exception while processing transaction: revert\n    at Function.RuntimeError.fromResults (/opt/ganache/resources/static/node/node_modules/ganache-core/lib/utils/runtimeerror.js:89:13)\n    at BlockchainDouble.processBlock (/opt/ganache/resources/static/node/node_modules/ganache-core/lib/blockchain_double.js:632:24)\n    at runMicrotasks (<anonymous>)\n    at processTicksAndRejections (internal/process/task_queues.js:93:5)', 'name': 'RuntimeError'}}

# Create a new loan

In [268]:
loaner
# Create a new loan
if index != -1:
    loanieType = accountsConract.functions.getType(loanerIndex).call()
    if loanieType:
        if createLoan(_loanie=web3.eth.accounts[2], _loaner=loaner, _amount=random.randint(1,100000), _organizationContract=organizationContract):
            print("Loan created")
else:
    print("This account is not registered in our system.")

'0x37594b8C8aC1D94457C86e30dE793099CE659817'

Enter the password of the organization: ········
Loan created
